# **1. Importar paquetes**

In [1]:
# 1. Importar funciones creadas en archivos Python, ubicadas en la carpeta Script/Python **Siempre que sea modificada, hay que reiniciar el kernel del notebook**
import sys
sys.path.insert(0, 'Script/python')

from lectura_archivos import lectura_todoscsv                   # Función que permite la lectura de todos los archivos, así trabajar todo en un mismo dataframe
from series_de_tiempo import series_de_tiempo                   # Función que permite la creación de series de tiempo
from ciclo_diario import ciclo_diario                           # Función que permite la creación de ciclos diarios
from ciclo_diario_direccion import ciclo_diario_direccion       # Función que permite la creación de ciclos diarios considerando la dirección del viento
from ciclo_estacional_viento import ciclo_estacional_viento     # Función que permite la creación de ciclos estacionales de la velocidad del viento
from ciclo_estacional import ciclo_estacional                   # Función que permite la creación de ciclos estacionales para cualquier otra variable
from rosa_viento import rosa_vientos                            # Función que permite la creación de rosas de los vientos en 8 y 16 direcciones

In [2]:
# 2. Importar paquetes necesarios 
from glob import glob                                   # Función que permite encontrar ruta (path) de cada archivo objetivo
import matplotlib.dates as mdates                       # Función que permite un manejo con los formatos de las fechas
import pandas as pd                                     # Función que permite el manejo de dataframes
import numpy as np                                      # Función que permite crear y manejar vectores y matrices de varias dimensiones
pd.options.display.float_format = '{:,.3f}'.format      # Opción para modificar la cantidad de decimales con las que se muestra los valores en el notebook.

In [3]:
# 3. Declaración de variables y ruta de cada archivo, apertura de archivos para crear variable que contendra todos los dataframes (df)
UTM_N        = 6171746
UTM_E        = 317503
Huso_horario = 19
EPSG         = "EPSG:9155" #SIRGAS-Chile 2016 / UTM zone 19S https://epsg.io/9155
paths        = sorted(glob("Data/Sfdo*.csv"))
df           = lectura_todoscsv(paths)
display(df.head())
display(df.tail())

,HR,MP10Hr,MP25Hr,OzoneHr,Pressure,Rad,Temp,VientoDir,VientoVel
Fecha,,,,,,,,,
2007-03-31 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-03-31 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-03-31 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-03-31 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-03-31 05:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,HR,MP10Hr,MP25Hr,OzoneHr,Pressure,Rad,Temp,VientoDir,VientoVel
Fecha,,,,,,,,,
2021-04-21 19:00:00,60.481,NaN,NaN,NaN,546.342,2.685,15.882,189.472,0.403
2021-04-21 20:00:00,69.174,NaN,NaN,NaN,559.475,0.127,14.547,160.301,1.423
2021-04-21 21:00:00,77.277,NaN,NaN,NaN,582.800,0.126,13.193,171.342,1.234
2021-04-21 22:00:00,79.993,NaN,NaN,NaN,595.833,0.161,12.433,208.226,0.934
2021-04-21 23:00:00,83.608,NaN,NaN,NaN,609.550,0.930,11.408,NaN,NaN


In [4]:
# 4. Información relacionada a la cantidad de (filas, columnas).
df.shape

(123263, 9)

In [5]:
# 5. Información del archivo, describe: número de filas, nombre de columnas, archivos no nulos, tipo de variable asignada a cada columna y memoria usada por la variable df (dataframe).
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 123263 entries, 2007-03-31 01:00:00 to 2021-04-21 23:00:00
Data columns (total 9 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   HR         69662 non-null   float32
 1   MP10Hr     58635 non-null   float32
 2   MP25Hr     31491 non-null   float32
 3   OzoneHr    68196 non-null   float32
 4   Pressure   65194 non-null   float32
 5   Rad        69660 non-null   float32
 6   Temp       67481 non-null   float32
 7   VientoDir  121563 non-null  float32
 8   VientoVel  122014 non-null  float32
dtypes: float32(9)
memory usage: 5.2 MB


In [6]:
# 6. Resumen estadístico de cada una de las variables, se incluye los percentiles 5% y 95% (los cuales no son por defecto)
df.describe(percentiles=[.05, .25, .50, .75, .95]).T
df.describe(percentiles=[.05, .25, .50, .75, .95]).T.to_excel("Output/Data/resumen_estadistico.xlsx")

# **2. Gráficos**

## 2.1 Series de tiempo

In [7]:
contenedor_de_limites = {
     "HR"         : [0, 100,  "Humedad relativa del aire (%)"],
     "MP10Hr"     : [0, 1000, "Material Particulado MP10 (μg/m3N)"],
     "MP25Hr"     : [0, 350,  "Material Particulado MP2.5 (μg/m3)"],
     "OzoneHr"    : [0, 100,  "Ozono (ppb)"],
     "Rad"        : [0, 1200, "Radiación global (W/m2)"],
     "Temp"       : [-5, 40,  "Temperatura ambiente (°C)"],
     "VientoDir"  : [0, 360,  "Dirección del viento (°)"],
     "VientoVel"  : [0, 8,    "Velocidad del viento (m/s)"],
     }
for variable, [vmin, vmax, var_title] in contenedor_de_limites.items():
    series_de_tiempo(
        df              = df, 
        variable        = variable, 
        nombre_estacion = "Estación San Fernando",
        xlabel          = "Tiempo [años]",
        ylabel          = var_title,
        ylim            = {"bottom": vmin, "top": vmax},
        major_locator   = {"locator": mdates.YearLocator()},
        major_formatter = {"formatter": mdates.DateFormatter('%Y')}
        )

## 2.2. Ciclos diarios

In [8]:
contenedor_de_limites = {
     "HR"         : [0, 100, "Humedad relativa del aire (%)"],
     "MP10Hr"     : [0, 200, "Material Particulado MP10 (μg/m3N)"],
     "MP25Hr"     : [0, 120,  "Material Particulado MP2.5 (μg/m3)"],
     "OzoneHr"    : [0, 60,  "Ozono (ppb)"],
     "Rad"        : [0, 1000, "Radiación global (W/m2)"],
     "Temp"       : [0, 40,  "Temperatura ambiente (°C)"],
     "VientoVel"  : [0, 5,   "Velocidad del viento (m/s)"],
     }

for variable, [vmin, vmax, var_title] in contenedor_de_limites.items():
    ciclo_diario(
        df              = df, 
        variable        = variable,  
        nombre_estacion = "Estación San Fernando",     
        xlabel          = "Hora Local [horas]",
        ylabel          = var_title,
        ylim            = {"bottom": vmin, "top": vmax},
        )

In [10]:
ciclo_diario_direccion(
    df              = df, 
    variable        = "VientoDir", 
    nombre_estacion = "Estación San Fernando",
    vmin            = 0,
    vmax            = 30
    )

## 2.3 Ciclos estacionales

In [11]:
ciclo_estacional_viento(
    df              = df, 
    velocidad       = "VientoVel", 
    direccion       = "VientoDir", 
    nombre_estacion = "Estación San Fernando",
    vmin            = 0,
    vmax            = 3,
    step            = 0.05,
    )

In [12]:
contenedor_de_limites = {
   "HR"       : [0, 100,  2.0, "Humedad relativa del aire",  "(%)"],
   "MP10Hr"   : [0, 150,  5.0, "Material Particulado MP10",  "(μg/m3N)"],
   "MP25Hr"   : [0, 100,  5.0, "Material Particulado MP2.5", "(μg/m3)"],
   "OzoneHr"  : [0, 50,   1.0, "Ozono", "(ppb)"],
   "Rad"      : [0, 1000,  50, "Radiación global", "(W/m2)"],
   "Temp"     : [0, 30,   1.0, "Temperatura ambiente", "(°C)"],
     }
for variable, [vmin, vmax, step, var_title, unit] in contenedor_de_limites.items():
     ciclo_estacional(
          df              = df, 
          variable        = variable, 
          nombre_estacion = "Estación San Fernando",
          vmin            = vmin,
          vmax            = vmax,
          step            = step,
          clabel          = var_title,
          unidad          = unit,
        )

## 2.4 Rosa de los vientos

In [13]:
rosa_vientos(
    df              = df, 
    var_vientos     = "VientoVel", 
    var_direccion   = "VientoDir", 
    nrosa           = 16, 
    nombre_estacion = "Estación San Fernando",
    )